In [6]:
! pip install scikit-learn

     ---------------------------------------- 8.2/8.2 MB 5.2 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 9.3 MB/s eta 0:00:00


In [25]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
import math
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer

# Function

In [8]:
def __is_bounded(direction,range,index,tokens_leng):
    cover = range*direction
    if cover+index<0 or cover+index >= tokens_leng:
        return True
    else:
        return False

In [9]:
def get_context(tokens, window_size):
    context_pair = []
    for i, token in enumerate(tokens):
        for j in range(1, window_size+1):
            if not __is_bounded(1,j,i,len(tokens)):
                context_pair.append((tokens[i],tokens[i+j]))
            if not __is_bounded(-1,j,i,len(tokens)):
                context_pair.append((tokens[i],tokens[i-j]))
    return context_pair

In [10]:
def __get_word_set(tokens):
    word_set = set()
    for token in tokens:
        word_set.add(token)
    return word_set

In [11]:
def __get_word_index(word_set):
    word_index_dic = dict()
    inverse_word_dic = dict()
    for i,word in enumerate(word_set):
        word_index_dic[word] = i
        inverse_word_dic[i] = word
    return word_index_dic,inverse_word_dic

# word2Vec

### 讀檔

In [12]:
# def chinese_word_cut(mytext):
#     return " ".join(jieba.cut(mytext))

df = pd.read_csv("Data/raw_data/review_clean_res_att/0.csv")
# df['cutted_comment'] = df.loc[:,"review_text"].apply(chinese_word_cut)

In [13]:
df.loc[:3,"review_text"]

0    北投巷弄竟然有那麼日式的拉麵，厲害，叉燒肥但非常入味，瘦肉點都不柴，對愛吃肥點的人根本是極品...
1      黑曜拉麵  蒜頭味很重的雞湯 配上撒的蒜酥 香氣十足  肉是控肉不是叉燒 整題吃下來有點太油了
2    晚上七點半左右到訪，並沒有其他客人店內有位阿嬤跟位較年輕的先生點餐前問了下+60豪華版是什麼...
3    身為住附近的拉麵宅終於來吃了  本來期望不高的  但實際來用餐  發現餐點的品質都不錯  拉...
Name: review_text, dtype: object

### 去除停用詞及標點符號

In [16]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding="gb18030") as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [17]:
poun=('，','。','！','：','「','」','…','、','？','【','】','.',':','?',';','!','~','`','+','-','<','>','/','[',']','{','}',"'",'"', '\n'," ","")

In [18]:
stop_words_file = "中文停用词库.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [19]:
list_ = []
for originReview in df.loc[:,"review_text"]:
    # 刪除標點符號
    filteredText = "".join(c for c in originReview if c not in (poun))
    # 讀入停用詞
    stopWords = {}.fromkeys([ line.rstrip() for line in stopwords])
    #  以精確模式分詞
    words = jieba.cut(filteredText, cut_all=False)
    # 去除停用詞後, 以空白斷詞
    final = ''
    for seg in words:
        if seg not in stopWords:
            seg+=' '
            final += seg
    list_.append(final)
df['cutted_comment'] = list_

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MASON8~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.532 seconds.
Prefix dict has been built successfully.


In [20]:
df['cutted_comment']

0      北投 巷弄 竟然 那麼 日式 拉 麵 厲害 叉 燒肥 非常 入味 瘦肉 點都 柴 對 愛吃 ...
1      黑曜拉 麵 蒜 頭味 很重 雞 湯配 撒 蒜 酥 香氣 十足 肉 控肉 叉燒 整題 吃 下來...
2      晚上 七點 半左右 訪並 沒有 客人 店內 有位 阿嬤跟位 較 年 輕 先生 點餐 前問 6...
3      身為 住 附近 拉 麵 宅終 於 來 吃 本來 期望 不高 實際來 用餐 發現 餐點 品質 ...
4      鹹 油 吃 完覺 腎臟 減少 20 年 使用 年限 真的 超級 鹹 回覆 店家 算術 不錯 ...
                             ...                        
210    CP 值低 餐點 好吃 價格 高 燙 青菜 很小 盤 50 元 周邊 餐廳 相比 不太 合理...
211    價格 親民 味道 很棒 拉 麵 湯較 少 偏重 口味 點整 體來 說 好吃 份量 多點點 更 讚 
212                                 平價 好吃 拉 麵 推叉 燒肉 必 吃 
213    黑曜拉 麵 麵 好吃 湯 好喝 油量 調整 下層 厚厚的 油太多 湯有點 鹹 叉 燒是 豬腩...
214       黑曜拉 麵 黑 蒜 頭 製 成 湯 頭 偏 淡 甜 沒 什 麼 蒜 味 黑蒜頭 獨有 甜味 
Name: cutted_comment, Length: 215, dtype: object

### 語料庫(Corpus)

In [21]:
text = df.loc[:,"cutted_comment"]
# 看前後文的時候，會看到兩格遠的詞
window_size = 2
# 代表學習到的Vector將會是長度為5的Vector
embedding_size = 5

if __name__ == '__main__':
    context_pair=[]
    word_set = set()

    for sentence in text:
        tokens = sentence.split(' ')
        context_pair += get_context(tokens,window_size)
        tmp_word_set = __get_word_set(tokens)
        for word in tmp_word_set:
            word_set.add(word)     

### 轉成One Hot形式 取得unique的詞彙數

In [22]:
word_index_dic,inverse_word_dic=__get_word_index(word_set)
word_size = len(word_set)
batch_size = len(context_pair)
print(word_index_dic)
print(inverse_word_dic)
print(batch_size)
print(word_size)

{'': 0, '應湯': 1, '迷幻': 2, '突出': 3, '燒感覺': 4, '評分個': 5, '看起': 6, '豪華版片': 7, '這些': 8, '跟湯': 9, '飽大': 10, '味感覺': 11, '所用': 12, '自行': 13, '油些': 14, '送員': 15, '還沒見': 16, '良心': 17, '幹': 18, '藉口': 19, '不要': 20, '容易': 21, '濃郁豚': 22, '短': 23, '裡吧': 24, '簡樸': 25, '蒜頭': 26, '熱想': 27, '粗度': 28, '天知道': 29, '實': 30, '重整': 31, '加點': 32, '太像': 33, '辣油': 34, '濃稠感': 35, '加湯': 36, '分量': 37, '態度': 38, '湯偏': 39, '台式': 40, '升級': 41, '輩': 42, '包': 43, '比較': 44, '上層': 45, '會放些': 46, '換換': 47, '吞': 48, '燒過': 49, '為目': 50, '將湯': 51, '純正': 52, '誘人': 53, '嬤': 54, '麻辣': 55, '真的': 56, '別': 57, '周邊': 58, '培根': 59, '需要': 60, '捲': 61, '雞': 62, '貪': 63, '儘': 64, '精神': 65, '客人': 66, '好椒': 67, '混亂': 68, '乾': 69, '負擔': 70, '平台': 71, '抱怨': 72, '失望': 73, '非常': 74, '飽滿': 75, '手工': 76, '不及': 77, '油': 78, '適合': 79, '張是': 80, '評斷': 81, '鞋蹺': 82, '第次': 83, '選擇': 84, '豬肉': 85, '噁': 86, '兩塊': 87, '花零錢': 88, '落實': 89, '更加': 90, '還沒有': 91, '人來': 92, '驚艷': 93, '上豚': 94, '天氣熱': 95, '進湯裡': 96, '熊本': 97, '指導先': 98, 'uber': 99, '給得': 100

### 將Context_pair裡面的input跟output分別取出並進行編碼

In [23]:
inputs = [word_index_dic[x[0]] for x in context_pair]
labels = [word_index_dic[x[1]] for x in context_pair]
print(inputs)
print(labels)

[523, 523, 1279, 1279, 1279, 1404, 1404, 1404, 1404, 1291, 1291, 1291, 1291, 1379, 1379, 1379, 1379, 2315, 2315, 2315, 2315, 2139, 2139, 2139, 2139, 851, 851, 851, 851, 138, 138, 138, 138, 1572, 1572, 1572, 1572, 74, 74, 74, 74, 857, 857, 857, 857, 1529, 1529, 1529, 1529, 731, 731, 731, 731, 2172, 2172, 2172, 2172, 1668, 1668, 1668, 1668, 1378, 1378, 1378, 1378, 1159, 1159, 1159, 1159, 822, 822, 822, 822, 1104, 1104, 1104, 1104, 74, 74, 74, 74, 1901, 1901, 1901, 1901, 1290, 1290, 1290, 1290, 74, 74, 74, 74, 1617, 1617, 1617, 1617, 2139, 2139, 2139, 2139, 1865, 1865, 1865, 1865, 787, 787, 787, 787, 1478, 1478, 1478, 1478, 1505, 1505, 1505, 1505, 1721, 1721, 1721, 1721, 749, 749, 749, 749, 1725, 1725, 1725, 1725, 238, 238, 238, 238, 2088, 2088, 2088, 2088, 861, 861, 861, 861, 1291, 1291, 1291, 1291, 1733, 1733, 1733, 1733, 1379, 1379, 1379, 1379, 1290, 1290, 1290, 1290, 456, 456, 456, 456, 32, 32, 32, 32, 356, 356, 356, 356, 2145, 2145, 2145, 2145, 424, 424, 424, 424, 955, 955, 955, 955,

### 資料放置的地方，他跟變數不同 batch_size就是全部有的training_data數

In [26]:
tf.disable_v2_behavior()
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size,])

Instructions for updating:
non-resource variables are not supported in the long term


### Projection Layer也就是word embedding的地方，其中的值先隨機從-1.0到1.0生成

In [27]:
embeddings = tf.Variable(
    tf.random_uniform([word_size, embedding_size], -1.0, 1.0))

### Input Layer跟Projection Layer的矩陣相乘變成查找

In [28]:
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

### nce_weights是Hidden Layer 跟 Output Layer 中間的Weight

In [29]:
nce_weights = tf.Variable(
    tf.truncated_normal([word_size,embedding_size],
                        stddev=1.0 / math.sqrt(embedding_size)))

nce_biases = tf.Variable(tf.zeros([word_size]))

### 繼續地往前傳遞，將embed乘上weight加上bias，就是我們的輸出了

In [30]:
prediction = tf.add(tf.matmul(embed, tf.transpose(nce_weights)), nce_biases)

### 使用tf提供的one_hot function來把output從編碼轉向量。

In [31]:
train_labels_vector = tf.one_hot(train_labels,word_size)

### 運算誤差 loss function

In [36]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=train_labels_vector))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 設置好Optimizer 跟session準備進行放資料訓練

In [37]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)
session = tf.Session()

### tf.variable初始化

In [38]:
init = tf.global_variables_initializer()
session.run(init)

In [39]:
for iteration in range(0,10000):
    total_loss = 0

    feed_dict = {train_inputs: inputs, train_labels: labels}
    _, cur_loss,pred= session.run([optimizer, loss, prediction], feed_dict=feed_dict)
    print('%s: loss: %s' %(iteration,cur_loss))

0: loss: 7.945714
1: loss: 7.941014
2: loss: 7.9363194
3: loss: 7.9316316
4: loss: 7.9269524
5: loss: 7.9222794
6: loss: 7.917613
7: loss: 7.912955
8: loss: 7.908304
9: loss: 7.90366
10: loss: 7.8990226
11: loss: 7.894394


KeyboardInterrupt: 

In [ ]:
tf.__version__

In [ ]:
! pip install tensorflow-gpu

In [ ]:
tmp = np.random.normal(0, 1, (3, 3))
tmp = tmp.astype(np.float32)
tmp.dtype
tmp = tf.constant(tmp)
tmp.device